In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import math

In [2]:
def conv_to_char(n):
    s = list(n)
    st = ""
    for i in s:
        st += chr(i)
    return st

In [3]:
def features_of_images_train_val(location_folder):
    
    list_of_files = os.listdir(location_folder)
    
    full_file_path = []
    for i in list_of_files:
        full_file_path.append(location_folder+i)

    id = []
    cls = []
    image = []

    for i in full_file_path:
        raw_img = tf.data.TFRecordDataset(i)
        image_description = {
            "id" : tf.io.FixedLenFeature([],tf.string),
            "class": tf.io.FixedLenFeature([],tf.int64),
            "image": tf.io.FixedLenFeature([],tf.string)
        }
        y = lambda x: tf.io.parse_single_example(x,image_description)
        img = raw_img.map(y)

        begin = i.rindex("-")+1
        end = i.index(".tfrec")
        num = int(i[begin:end])

        for dr in img.take(num):
            id.append(conv_to_char(dr["id"].numpy()))
            cls.append(int(dr["class"].numpy()))
            image.append(tf.io.decode_image(dr["image"]).numpy())
    
    id = np.array(id)
    cls = np.array(cls)
    image = np.array(image)
    
    return id,cls,image

In [4]:
def features_of_images_test(location_folder):

    list_of_files = os.listdir(location_folder)

    full_file_path = []
    for i in list_of_files:
        full_file_path.append(location_folder+i)

    id = []
    image = []

    for i in full_file_path:
        raw_img = tf.data.TFRecordDataset(i)
        image_description = {
            "id" : tf.io.FixedLenFeature([],tf.string),
            "image": tf.io.FixedLenFeature([],tf.string)
        }
        y = lambda x: tf.io.parse_single_example(x,image_description)
        img = raw_img.map(y)

        begin = i.rindex("-")+1
        end = i.index(".tfrec")
        num = int(i[begin:end])

        for dr in img.take(num):
            id.append(conv_to_char(dr["id"].numpy()))
            image.append(tf.io.decode_image(dr["image"]).numpy())

    id = np.array(id)
    image = np.array(image)
    
    return id,image

In [5]:
location_folder_train_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/"

location_folder_train_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/train/"

location_folder_train_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/train/"

location_folder_train_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/train/"

In [6]:
location_folder_val_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/val/"

location_folder_val_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/val/"

location_folder_val_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/val/"

location_folder_val_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/val/"

In [7]:
location_folder_test_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/test/"

location_folder_test_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/test/"

location_folder_test_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/test/"

location_folder_test_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/test/"

### Training Dataset

In [8]:
id_train_192,cls_train_192,image_train_192 = features_of_images_train_val(location_folder_train_192)

In [9]:
id_train_224,cls_train_224,image_train_224 = features_of_images_train_val(location_folder_train_224)

In [10]:
id_train_331,cls_train_331,image_train_331 = features_of_images_train_val(location_folder_train_331)

In [11]:
id_train_512,cls_train_512,image_train_512 = features_of_images_train_val(location_folder_train_512)

In [ ]:
i = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/train/00-512x512-798.tfrec"
raw_img = tf.data.TFRecordDataset(i)
image_description = {
    "id" : tf.io.FixedLenFeature([],tf.string),
    "image": tf.io.FixedLenFeature([],tf.string)
}
y = lambda x: tf.io.parse_single_example(x,image_description)
img = raw_img.map(y)

begin = i.rindex("-")+1
end = i.index(".tfrec")
num = int(i[begin:end])
id = []
image = []

for dr in img.take(num):
    id.append(conv_to_char(dr["id"].numpy()))
    image.append(tf.io.decode_image(dr["image"].numpy(),dtype="float64"))

id = np.array(id)
image = np.array(image)

In [52]:
image

[<tf.Tensor: shape=(512, 512, 3), dtype=uint8, numpy=
 array([[[14, 22,  9],
         [14, 20,  6],
         [16, 20,  5],
         ...,
         [26, 28, 14],
         [26, 28, 14],
         [26, 28, 14]],
 
        [[14, 22,  7],
         [14, 20,  6],
         [16, 20,  5],
         ...,
         [26, 28, 14],
         [27, 29, 15],
         [28, 30, 16]],
 
        [[14, 22,  7],
         [15, 21,  7],
         [16, 20,  5],
         ...,
         [26, 28, 14],
         [27, 29, 15],
         [29, 31, 17]],
 
        ...,
 
        [[26, 34, 10],
         [26, 34, 10],
         [25, 34,  7],
         ...,
         [29, 40,  6],
         [28, 39,  9],
         [27, 35, 11]],
 
        [[26, 34, 10],
         [25, 33,  9],
         [25, 34,  7],
         ...,
         [28, 42,  7],
         [30, 41, 11],
         [29, 39, 14]],
 
        [[26, 34, 10],
         [25, 33,  9],
         [25, 33,  9],
         ...,
         [28, 42,  7],
         [28, 41, 11],
         [27, 40, 12]]], dt

### Validation Dataset

In [ ]:
id_val_192,cls_val_192,image_val_192 = features_of_images_train_val(location_folder_val_192)

In [ ]:
id_val_224,cls_val_224,image_val_224 = features_of_images_train_val(location_folder_val_224)

In [ ]:
id_val_331,cls_val_331,image_val_331 = features_of_images_train_val(location_folder_val_331)

In [ ]:
id_val_512,cls_val_512,image_val_512 = features_of_images_train_val(location_folder_val_512)

### Testing Dataset

In [ ]:
id_test_192,image_test_192 = features_of_images_test(location_folder_test_192)

In [ ]:
id_test_224,image_test_224 = features_of_images_test(location_folder_test_224)

In [ ]:
id_test_331,image_test_331 = features_of_images_test(location_folder_test_331)

In [ ]:
id_test_512,image_test_512 = features_of_images_test(location_folder_test_512)

### Deep Learning

In [44]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Resizing(height=512,width=512),
    tf.keras.layers.Conv2D(filters=64,kernel_size=4,activation="relu",input_shape=(512,512,3)),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=128,kernel_size=8,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=256,kernel_size=16,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=128,kernel_size=8,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Conv2D(filters=64,kernel_size=4,activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=4),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=128,activation="relu"),
    tf.keras.layers.Dense(units=104,activation="softmax")
])

In [45]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.RMSprop(),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=tf.keras.metrics.categorical_accuracy,
)

In [46]:
model.fit(
    x=image_train_192,
    y=cls_train_192,
    batch_size=10,
    epochs=100,
    verbose=2,
    callbacks=tf.keras.callbacks.EarlyStopping(patience=2)
)

Epoch 1/100


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_t